In [2]:
import sys
import os
import json
import pandas
import numpy
import optparse
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict
from sklearn.model_selection import train_test_split 

In [60]:
requests_data_raw = pandas.read_csv('Datasets/output_http_csic_2010_full.csv')
requests_data_raw.head(5)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,index,method,url,protocol,userAgent,pragma,cacheControl,accept,acceptEncoding,acceptCharset,acceptLanguage,host,connection,contentLength,contentType,cookie,payload,label
0,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,id=2,anom
1,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,nombre=Jam%F3n+Ib%E9rico,anom
2,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,precio=85,anom
3,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,cantidad=%27%3B+DROP+TABLE+usuarios%3B+SELECT+...,anom
4,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,B1=A%F1adir+al+carrito,anom


In [61]:
# Drop unnecessary columns
requests_data = requests_data_raw.drop(['index', 'contentLength', 'cacheControl'], axis=1)

# Rename Country names to match temperatures dataset
requests_data['label'] = requests_data['label'].replace(
                                        {'anom': 1,
                                         'norm': 0
                                        })

# Covert NaN columns to empty string
requests_data.fillna('', inplace=True)
requests_data.head(5)

,method,url,protocol,userAgent,pragma,accept,acceptEncoding,acceptCharset,acceptLanguage,host,connection,contentType,cookie,payload,label
0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,id=2,1
1,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,nombre=Jam%F3n+Ib%E9rico,1
2,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,precio=85,1
3,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,cantidad=%27%3B+DROP+TABLE+usuarios%3B+SELECT+...,1
4,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,B1=A%F1adir+al+carrito,1


In [62]:
dataset = requests_data.sample(frac=1).values # convert data into array
dataset

array([['POST', 'http://localhost:8080/tienda1/publico/autenticar.jsp',
        'HTTP/1.1', ..., 'JSESSIONID=20A1C3D5826A924F58F82F72DD4F2981',
        'remember=off', 0],
       ['GET', 'http://localhost:8080/tienda1/publico/registro.jsp',
        'HTTP/1.1', ..., 'JSESSIONID=816D20687DCF87A634E45AAA70C047CE',
        'provincia=Ja%E9n', 1],
       ['POST', 'http://localhost:8080/tienda1/publico/registro.jsp',
        'HTTP/1.1', ..., 'JSESSIONID=27BDA052571222A449015881D93BC6B4',
        'modo=registro', 0],
       ...,
       ['POST', 'http://localhost:8080/tienda1/publico/vaciar.jsp',
        'HTTP/1.1', ..., 'JSESSIONID=F4B06A4A7286FBD8A396947918C2FA83',
        'B2=Vaciar+carritosessionid%3D12312312%26+username%3D%253C%2573%2563%2572%2569%2570%2574%253E%2564%256F%2563%2575%256D%2565%256E%2574%252E%256C%256F%2563%2561%2574%2569%256F%256E%253D%2527%2568%2574%2574%2570%253A%252F%252F%2561%2574%2574%2561%2563%256B%2565%2572%2568%256F%2573%2574%252E%2565%2578%2561%256D%2570%256C%2565%

In [63]:
# Preprocess dataset
X = dataset[:,:14]
X

array([['POST', 'http://localhost:8080/tienda1/publico/autenticar.jsp',
        'HTTP/1.1', ..., 'application/x-www-form-urlencoded',
        'JSESSIONID=20A1C3D5826A924F58F82F72DD4F2981', 'remember=off'],
       ['GET', 'http://localhost:8080/tienda1/publico/registro.jsp',
        'HTTP/1.1', ..., '',
        'JSESSIONID=816D20687DCF87A634E45AAA70C047CE',
        'provincia=Ja%E9n'],
       ['POST', 'http://localhost:8080/tienda1/publico/registro.jsp',
        'HTTP/1.1', ..., 'application/x-www-form-urlencoded',
        'JSESSIONID=27BDA052571222A449015881D93BC6B4', 'modo=registro'],
       ...,
       ['POST', 'http://localhost:8080/tienda1/publico/vaciar.jsp',
        'HTTP/1.1', ..., 'application/x-www-form-urlencoded',
        'JSESSIONID=F4B06A4A7286FBD8A396947918C2FA83',
        'B2=Vaciar+carritosessionid%3D12312312%26+username%3D%253C%2573%2563%2572%2569%2570%2574%253E%2564%256F%2563%2575%256D%2565%256E%2574%252E%256C%256F%2563%2561%2574%2569%256F%256E%253D%2527%2568%2574%257

In [64]:
Y = dataset[:,14]
Y

array([0, 1, 0, ..., 1, 1, 1], dtype=object)

In [65]:
tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(X)

In [66]:
tokenizer.word_index

{'': 10,
 'email=venohr%40devuelos.pg': 28257,
 'JSESSIONID=A430951D6B2726464C237CCCBA52BD56': 94731,
 'JSESSIONID=D5BB0586B38815900DBB0EC814C6F251': 69647,
 'JSESSIONID=4BCE8F80074FAF421200E483149FCAA6': 58649,
 'JSESSIONID=979C6565A8A9184CC9A4EC4336DAC926': 61692,
 'dni=88301886X': 29653,
 'JSESSIONID=A8A6087B5A253A3E3C2FAEBEDCA1CD13': 95883,
 'JSESSIONID=BEF622073BDFE1BFC4ACBE3F792B4F61': 72768,
 'JSESSIONID=86AA46DB85D5C2AC8AFE9DBDD71FE4FC': 66952,
 'JSESSIONID=8F24D23061B8C39FDD76F9AE1C966D3C': 92630,
 'JSESSIONID=26EDB56A37162DC50E1BF26890E93E76': 79155,
 'JSESSIONID=A8ED1A0E8A9579F17B9862B9E643D975': 24787,
 'ntc=5170755929650123': 47425,
 'password=-saborete': 38656,
 'JSESSIONID=3FE74A4E2771D3431803E0B6F4FF1EB5': 66137,
 'dni=36455660Q': 21459,
 'JSESSIONID=A2290F8555672BA3042D3D6B8BF04191': 88171,
 'JSESSIONID=573884E9DF9FF2ECF52C9EFB1E608A7D': 75119,
 'JSESSIONID=26E0E3D58FA868281C6D4423513EF67C': 63569,
 'JSESSIONID=3CF1BC82050D029F3A86B08CA8DF2218': 20014,
 'password=SoPor

In [67]:
# Extract and save word dictionary
word_dict_file = 'build/word-dictionary.json'

In [68]:
if not os.path.exists(os.path.dirname(word_dict_file)):
    os.makedirs(os.path.dirname(word_dict_file))

with open(word_dict_file, 'w') as outfile:
    json.dump(tokenizer.word_index, outfile, ensure_ascii=False)

In [69]:
num_words = len(tokenizer.word_index)+1
num_words

96493

In [70]:
X = tokenizer.texts_to_sequences(X)
X

[[13, 17, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 16676, 35],
 [11, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 519, 107],
 [13, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 520, 20],
 [13, 27, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 58591, 10115],
 [13, 26, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 58592, 33],
 [13, 17, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 16677, 23],
 [11, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 521, 72],
 [11, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 522, 10899],
 [13, 15, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 523, 10900],
 [11, 27, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 58593, 66],
 [13, 15, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 524, 26719],
 [13, 16, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 16678, 42],
 [11, 45, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 58594, 10],
 [13, 15, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 525, 37859],
 [13, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 526, 37860],
 [13, 15, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 527, 37861],
 [11, 46, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 58595, 10],
 [11, 17, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 16679, 26720],
 [13, 15, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12

In [71]:
# split data into training set and a temporary set using sklearn.model_selection.traing_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=23)  

In [72]:
X_train
#X_train = numpy.array(X_train)
#X_train

[[13, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 1232, 27597],
 [11, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 547, 15692],
 [11, 29331, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 63526, 10],
 [13, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 1287, 31010],
 [11, 16, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 19372, 21],
 [11, 15, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 4905, 20],
 [11, 16, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 23842, 44],
 [11, 15, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 2075, 32438],
 [13, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 6413, 10793],
 [11, 17, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 24787, 55890],
 [11, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 1712, 42721],
 [11, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 1999, 30303],
 [13, 18, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 36732, 49452],
 [11, 15, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 694, 30562],
 [11, 15, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 2677, 28352],
 [11, 18, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 33896, 53496],
 [11, 14, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 993, 50053],
 [13, 16, 1, 2, 3, 4, 5, 6, 7, 9, 8, 12, 20728, 29101],
 [13, 17, 1, 2,

In [73]:
tb_callback = TensorBoard(log_dir='./logs', embeddings_freq=1)

In [74]:
model = Sequential()
model.add(Embedding(num_words, 32, input_length=14))
model.add(Dropout(0.5))
model.add(LSTM(64, recurrent_dropout=0.5))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(numpy.array(X_train), numpy.array(y_train), validation_split=0.25, epochs=3, batch_size=128, callbacks=[tb_callback])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 14, 32)            3087776   
_________________________________________________________________
dropout_11 (Dropout)         (None, 14, 32)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 3,112,673
Trainable params: 3,112,673
Non-trainable params: 0
_________________________________________________________________
None
Train on 134151 samples, validate on 44717 samples
Epoch 1/3
134151/134151 [==============================] - 105s 782us/step -

In [76]:
# Evaluate model
score, acc = model.evaluate(numpy.array(X_test), numpy.array(y_test), verbose=1, batch_size=128)

44717/44717 [==============================] - 2s 53us/step


In [77]:
print("Model Accuracy: {:0.2f}%".format(acc * 100))

Model Accuracy: 99.09%


In [78]:
# Save model
model.save_weights('model-weights.h5')
model.save('model.h5')
with open('json', 'w') as outfile:
    outfile.write(model.to_json())